# Traffic Detection Demo

# Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt

# This is needed to display the images.
%matplotlib inline

if tf.__version__ < '1.4.0':
  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')


## Object detection imports
Here are the imports from the object detection module.

In [2]:
# adding path for the tensorflow object detection API
sys.path.append("/home/zwlin/projects/tensorflow-related/models/research/object_detection")
from utils import label_map_util
from utils import visualization_utils as vis_util

# Model preparation 

In [3]:
# Path to frozen detection graph. This is the actual model that is used for the object detection.
#PATH_TO_CKPT = 'models/faster_rcnn_resnet101_traffic_lights/export/frozen_inference_graph.pb'
PATH_TO_CKPT = 'models/ssd_mobilenet_v1_traffic_lights/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'config/traffic_lights_label_map.pbtxt'

NUM_CLASSES = 1

## Load a (frozen) Tensorflow model into memory.

In [4]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `1`, we know that this corresponds to `traffic-lights`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [5]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# Detection

In [6]:
from glob import glob
import cv2

# get the testing images
PATH_TO_TEST_IMAGES_DIR = "input"#'data/rgb/test'
test_images = glob(PATH_TO_TEST_IMAGES_DIR + '/*.png')
test_images.sort()

# create a VideoWrite to write the detection result
image0 = cv2.imread(test_images[0])
image_size = (image0.shape[1], image0.shape[0])
fourcc=cv2.VideoWriter_fourcc(*"MJPG")
fps = 1
video_writer = cv2.VideoWriter('./test.avi', fourcc, fps, image_size)

In [7]:
from tqdm import tqdm
# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)
with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    # Definite input and output Tensors for detection_graph
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    # Each box represents a part of the image where a particular object was detected.
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    # Each score represent how level of confidence for each of the objects.
    # Score is shown on the result image, together with the class label.
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')
    for image_path in tqdm(test_images, desc='traffic light detecting', unit='images'):
      # Load image
      image_np = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
      # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
      image_np_expanded = np.expand_dims(image_np, axis=0)
      # Actual detection.
      (boxes, scores, classes, num) = sess.run(
          [detection_boxes, detection_scores, detection_classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})
      print("num = ", num, "len(boxes) = ", len(boxes[0]))
      # Draw bboxes for the results of a detection.
      vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          np.squeeze(boxes),
          np.squeeze(classes).astype(np.int32),
          np.squeeze(scores),
          category_index,
          use_normalized_coordinates=True,
          line_thickness=8)
      # plt.figure(figsize=IMAGE_SIZE)
      # plt.imshow(image_np)
      # Write the result to a video file
      video_writer.write(image_np)
        
video_writer.release()

traffic light detecting:  36%|███▋      | 4/11 [00:01<00:05,  1.39images/s]

num =  [100.] len(boxes) =  100
num =  [100.] len(boxes) =  100
num =  [100.] len(boxes) =  100
num =  [100.] len(boxes) =  100
num =  [100.] len(boxes) =  100


traffic light detecting:  82%|████████▏ | 9/11 [00:01<00:00,  2.63images/s]

num =  [100.] len(boxes) =  100
num =  [100.] len(boxes) =  100
num =  [100.] len(boxes) =  100
num =  [100.] len(boxes) =  100


traffic light detecting: 100%|██████████| 11/11 [00:01<00:00,  3.42images/s]

num =  [100.] len(boxes) =  100
num =  [100.] len(boxes) =  100
